# Using computer vision to detect roundabouts

In [1]:
import pandas as pd
import os
import numpy as np
import datashader as ds
import colorcet
from scipy.ndimage import gaussian_filter
import xarray as xr
from pyproj import Proj, transform
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from scipy.ndimage import convolve
root_path = 'probe_data/'
folders = [root_path + str(i) + "/" for i in range(15)]

def clean_the_df(df):
    df = df[df['speed'] > 20]
    return df
def read_folder(folder):
    dfs = []
    print(f'Reading folder: {folder}')
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder, filename)
            dfs.append(pd.read_csv(file_path))

    final_df = pd.concat(dfs, ignore_index=True)
    final_df = clean_the_df(final_df)
    return final_df

In [ ]:
dfs = [read_folder(folder) for folder in folders]

In [3]:
roundabouts_df = pd.read_csv('probe_data/hamburg_rounsabouts.csv')

### How to zoom or focus on a region from the data?

We use a sliding window to go through regions of the data; this enables us to focus on a small region and have better resolution. 
First, we subdivide the whole map of trace into a grid, after that we do a sliding window with window_size=2, 

note: a better way to do a sliding window is to avoid subdividing the whole dataframe into different dataframes to save memory.

In [4]:
def subdivide_df(df, subdivisions:int=7):
	min_long, min_lat, max_long, max_lat = (
		df['longitude'].min(),
		df['latitude'].min(),
		df['longitude'].max(),
		df['latitude'].max(),
	)

	# Calculate the step size for longitude and latitude based on subdivisions
	long_step = (max_long - min_long) / subdivisions
	lat_step = (max_lat - min_lat) / subdivisions

	# Create a list to store the subdivided dataframes
	subdivided_dfs = [[None for _ in range(subdivisions)] for _ in range(subdivisions)]


	# Loop through each subdivision and filter the dataframe based on grid
	for i in range(subdivisions):

		for j in range(subdivisions):
			# Define the longitude and latitude bounds for this subdivision
			long_min = min_long + i * long_step
			long_max = long_min + long_step
			lat_min = min_lat + j * lat_step
			lat_max = lat_min + lat_step
			
			# Filter the dataframe for rows that fall within these bounds
			sub_df = df[(df['longitude'] >= long_min) & (df['longitude'] < long_max) &
						(df['latitude'] >= lat_min) & (df['latitude'] < lat_max)]
			
			# Append the subdivided datafradme to the list
			subdivided_dfs[subdivisions - j - 1][i] = sub_df

	return subdivided_dfs
	
def sliding_window(dfs, window_size=2):
	rows, cols = len(dfs), len(dfs[0])

	result = [[None for _ in range(cols - window_size + 1)] for _ in range(rows - window_size + 1)]

	for i in range(rows - window_size + 1):
		for j in range(cols - window_size + 1):
			window = [dfs[i + k][j + l] for k in range(window_size) for l in range(window_size)]
			concatenated_df = pd.concat(window, axis=0)
			result[i][j] = concatenated_df

	return result

In [18]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def detect_contours(image, return_highlighted_image=True):

    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)

    inverted = cv2.bitwise_not(gray)
    # Apply thresholding to the inverted image
    _, binary = cv2.threshold(inverted, 70, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if return_highlighted_image:
        image_with_contours = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        cv2.drawContours(image_with_contours, contours, -1, (255, 0, 255), thickness=3)
        return contours, Image.fromarray(image_with_contours)
    else:
        return contours

def detect_circles(agg, min_area=100):
    image = ds.tf.shade(agg, cmap=colorcet.gray, how='log').to_pil()
    contours, _ = detect_contours(image)
    circle_centers = []

    # Draw the contours on the original image
    image_with_circles = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    for contour in contours:
        # Calculate perimeter and apply shape approximation
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

        # Calculate area and circularity
        area = cv2.contourArea(contour)

        # Filter out small areas
        if area < min_area:
            continue  # Skip small contours

        if perimeter == 0:
            continue
        circularity = 4 * np.pi * (area / (perimeter ** 2))

        # Detect circles based on circularity and number of vertices
        if len(approx) > 4 and circularity > 0.8:
            # Compute the center of the circle using moments
            M = cv2.moments(contour)
            if M["m00"] != 0:  # To avoid division by zero
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                circle_centers.append((cX, cY))

                cv2.drawContours(image_with_circles, [contour], -1, (0, 0, 255), thickness=3)
                cv2.circle(image_with_circles, (cX, cY), 5, (0, 255, 0), -1)

    return circle_centers, Image.fromarray(image_with_circles)




In [28]:
subdivisions = subdivide_df(dfs[0], 7)
windows = sliding_window(subdivisions)

In [24]:
def log_xarray(array):
    def safe_log(data):
        return np.log(data.clip(min=1e-10))
    return xr.DataArray(safe_log(array), dims=array.dims, coords=array.coords, attrs=array.attrs)
    
def remove_outliers(array):
	higher_threshold = array.quantile(.99) 
	filtered_data = array.where(array < higher_threshold, 0)
	return filtered_data

def aggregate_df(df, resolution=500):
	cvs = ds.Canvas(plot_height=resolution, plot_width=resolution, 
	)

	points_agg = cvs.points(df, 'longitude', 'latitude')

	cleaned_agg = remove_outliers(points_agg)
	# smooth the aggregate data to remove noise
	smoothed_points_agg = gaussian_filter(cleaned_agg.values, sigma=3)

	smoothed_points_agg = xr.DataArray(smoothed_points_agg, coords=cleaned_agg.coords, dims=cleaned_agg.dims)

	return smoothed_points_agg

In [26]:
def detect_roundabouts(df, draw_images=False):

    roundabouts = []

    # Aggregate data for this grid section
    agg = aggregate_df(df, 500)
    agg = log_xarray(agg)
    
    circles, circled_image = detect_circles(agg, 10)
    if draw_images or len(circles) != 0:
        display(circled_image)
    min_lon, max_lon = df['longitude'].min(), df['longitude'].max()
    agg_width, agg_height = agg.shape[1], agg.shape[0]

    min_lat, max_lat = df['latitude'].min(), df['latitude'].max()
    def can_add(roundsabouts, cx, cy):
        for rb in roundsabouts:
            pix, _ = rb
            cx2, cy2 = pix
            if (cx2 -cx) **2 + (cy2 - cy) **2 < 40:
                return False

        return True

    for cX, cY in circles:
        lon = min_lon + (cX / agg_width) * (max_lon - min_lon)
        lat = min_lat + (cY / agg_height) * (max_lat - min_lat)
        print(f"Detected roundabout center: Longitude={lon}, Latitude={lat}")
        if can_add(roundabouts, cX,cY):
            roundabouts.append(((cX,cY), (lon, lat)))
    return [lonlat for _, lonlat in roundabouts]

In [ ]:
for i, df in enumerate(dfs):
	print(f"BBox {i}")
	subdivisions = subdivide_df(df, 7);
	windows = sliding_window(subdivisions)
	grid_size = len(windows)
	
	for i in range(grid_size):
		for j in range(grid_size):
			current_df = windows[i][j]
			roundsabouts = detect_roundabouts(current_df)